# Generate Memes

## Instructions

In this project, we developed a deep learning architecture to generate new and creative memes. This notebook allows you to load an image and get a meme text written on it. The notebook is split into different sections for loading the model, processing the image and generating the meme. Please do not change the code unless you know what you are doing.

Have fun :)

## Load Model (Run once)

In [1]:
import numpy as np
from training.MergeModel import MergeModel
from training.Dataset import Dataset
from training.DatasetProcessor import DatasetProcessor
from testing.EvaluateModel import EvaluateModel
from preprocessing.EfficientNet import EfficientNet
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.efficientnet import preprocess_input

In [2]:
model_path = 'models/BestModel/B0'
model_name = 'Merge-model-B0-n256-ln256-tanhpTrue-ep004-loss4.717-val_loss5.467'
train_captions_path = 'dataset/CaptionsClean3_train.txt'
glove_path = 'embedding/glove/captionsGlove.pkl'
ip_core = 'B0'

In [3]:
model_obj = MergeModel(model_path, init=False, model_name=model_name + '.h5')
ip_obj = EfficientNet(None, models_list=(ip_core,))
ip_layer = ip_obj.init_models((ip_core,))

In [4]:
t_captions = DatasetProcessor(train_captions_path, None, None, glove_path)
t_captions.process_dataset(t_captions.captions_to_list())

## Process Image (Run for every image)

In [7]:
image_path = input('Enter the image path: ')
image_name = image_path.split('/')[-1].split('.')[0] + '-gen'
generated_memes_path = '/'.join(image_path.split('/')[:-1])
size = ip_layer[ip_core][1]
image = {image_name: load_img(image_path, target_size=(size, size))}
p_image = img_to_array(image[image_name])
p_image = np.expand_dims(p_image, axis=0)
p_image = preprocess_input(p_image)
image_feature = ip_layer[ip_core][0].predict(p_image, verbose=0)
caption = {image_name: EvaluateModel(model_obj, t_captions).beam_search(image_feature).split()}
Dataset().show_samples(images_dict=image, captions_dict=caption, num=1, randomise=False, 
                       splitted=True, output_file_path=generated_memes_path)